In [1]:
import tensorflow as tf
import numpy as np
import os
import zipfile
from tensorflow.keras import layers, models

2024-11-17 21:03:20.366037: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-17 21:03:20.377218: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1731906200.390883   91446 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1731906200.394755   91446 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-17 21:03:20.408505: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
print("Physical GPUs:", tf.config.list_physical_devices('GPU'))
print("Is GPU available:", tf.test.is_gpu_available())


Physical GPUs: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
Is GPU available: True


I0000 00:00:1731906201.656143   91446 gpu_device.cc:2022] Created device /device:GPU:0 with 4280 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3060 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6


# Load the dataset

In [3]:
def load_chorales(data_dir="jsb_chorales 2"):

    dataset = {}
    for subset in ['train', 'valid', 'test']:
        subset_path = os.path.join(data_dir, subset)
        chorales = []
        for file in sorted(os.listdir(subset_path)):
            if file.endswith(".csv"):
                filepath = os.path.join(subset_path, file)
                chorale = np.loadtxt(filepath, delimiter=",", skiprows=1, dtype=int)
                chorales.append(chorale)
        dataset[subset] = chorales
        print(f"Loaded {len(chorales)} chorales from {subset} directory.")
    return dataset

# Example usage
chorale_data = load_chorales()
train_chorales = chorale_data['train']
valid_chorales = chorale_data['valid']
test_chorales = chorale_data['test']


Loaded 229 chorales from train directory.
Loaded 76 chorales from valid directory.
Loaded 77 chorales from test directory.


In [4]:
# Find the max value for normalizing
max_value = max([chorale.max() for chorale in train_chorales])
max_value

np.int64(81)

# Create the dataset

In [5]:
# Prepare the data
def create_dataset(chorales, seq_length):
    X, y = [], []
    for chorale in chorales:
        for i in range(len(chorale) - seq_length):
            X.append(chorale[i:i + seq_length])
            y.append(chorale[i + seq_length])
    return np.array(X), np.array(y)

# Generate combinations of models to try

In [6]:
from itertools import product

# Define hyperparameter space
param_grid = {
    'rnn_units': [32, 64],
    'dense_units': [64, 128],
    'batch_size': [16, 32],
    'epochs': [10, 20]
}

# Generate all combinations of parameters
param_combinations = list(product(*param_grid.values()))


# Print all combinations
for i, params in enumerate(param_combinations, 1):
    print(f"Combination {i}: {params}")
 

Combination 1: (32, 64, 16, 10)
Combination 2: (32, 64, 16, 20)
Combination 3: (32, 64, 32, 10)
Combination 4: (32, 64, 32, 20)
Combination 5: (32, 128, 16, 10)
Combination 6: (32, 128, 16, 20)
Combination 7: (32, 128, 32, 10)
Combination 8: (32, 128, 32, 20)
Combination 9: (64, 64, 16, 10)
Combination 10: (64, 64, 16, 20)
Combination 11: (64, 64, 32, 10)
Combination 12: (64, 64, 32, 20)
Combination 13: (64, 128, 16, 10)
Combination 14: (64, 128, 16, 20)
Combination 15: (64, 128, 32, 10)
Combination 16: (64, 128, 32, 20)


# Build model function

In [7]:
def build_model(seq_length, input_dim, rnn_units, dense_units):

    inputs = layers.Input(shape=(seq_length, input_dim))
    x = layers.SimpleRNN(rnn_units, activation="tanh", return_sequences=False)(inputs)
    x = layers.Dense(dense_units, activation="relu")(x)
    outputs = layers.Dense(input_dim, activation="linear")(x)  # Linear activation for regression

    model = models.Model(inputs=inputs, outputs=outputs)
    
    # Compile the model
    model.compile(optimizer="adam", loss="mse", metrics=["mae"])
    return model



# Apply cross validation to all models to find the best combination of parameters.

In [8]:
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import KFold
import datetime

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

SEQ_LENGTH = 32  # Length of input sequences

X_train, y_train = create_dataset(train_chorales, SEQ_LENGTH)
X_valid, y_valid = create_dataset(valid_chorales, SEQ_LENGTH)
X_test, y_test = create_dataset(test_chorales, SEQ_LENGTH)

# Normalize and reshape data
X_train, X_valid, X_test = X_train / max_value, X_valid / max_value, X_test / max_value
y_train, y_valid, y_test = y_train / max_value, y_valid / max_value, y_test / max_value

early_stopping = EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)
kf = KFold(n_splits=5, shuffle=True, random_state=42)  # 5-fold cross-validation

fold_results = []
best_params = None
best_loss = float('inf')

for idx, params in enumerate(param_combinations):
    print(f"Testing parameter combination {idx + 1}/{len(param_combinations)}: {params}")
    
    # Extract hyperparameters for this combination
    rnn_units, dense_units, batch_size, epochs = params

    fold_losses = []
    
    for fold, (train_idx, val_idx) in enumerate(kf.split(X_train)):
        print(f"Training fold {fold + 1}/{kf.n_splits}...")
        
        X_fold_train, X_fold_val = X_train[train_idx], X_train[val_idx]
        y_fold_train, y_fold_val = y_train[train_idx], y_train[val_idx]
        
        # Build the model

        model = build_model(SEQ_LENGTH, X_train.shape[-1], rnn_units, dense_units)
        
        # Train the model
        history = model.fit(
            X_fold_train, y_fold_train,
            epochs=epochs,
            batch_size=batch_size,
            validation_data=(X_fold_val, y_fold_val),
            callbacks=[early_stopping, tensorboard_callback],
            verbose=0
        )
        
        # Save fold loss
        final_val_loss = history.history['val_loss'][-1]
        fold_losses.append(final_val_loss)
    
    # Calculate average validation loss across folds
    avg_val_loss = sum(fold_losses) / len(fold_losses)
    print(f"Parameter combination {idx + 1} average validation loss: {avg_val_loss:.4f} cross val losses: {fold_losses}")
    
    # Update best parameters if current loss is lower
    if avg_val_loss < best_loss:
        best_loss = avg_val_loss
        best_params = params

print(f"Best parameters: {best_params} with validation loss: {best_loss:.4f}")



Testing parameter combination 1/16: (32, 64, 16, 10)
Training fold 1/5...


I0000 00:00:1731906201.995130   91446 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 4280 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3060 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6
I0000 00:00:1731906203.454822   91542 service.cc:148] XLA service 0x71280c0051c0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1731906203.454838   91542 service.cc:156]   StreamExecutor device (0): NVIDIA GeForce RTX 3060 Laptop GPU, Compute Capability 8.6
2024-11-17 21:03:23.475005: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1731906203.545366   91542 cuda_dnn.cc:529] Loaded cuDNN version 90300
2024-11-17 21:03:23.575019: W external/local_xla/xla/service/gpu/nvptx_compiler.cc:930] The NVIDIA driver's CUDA version is 12.2 which is older than the PTX compiler version 12

Training fold 2/5...
Training fold 3/5...
Training fold 4/5...
Training fold 5/5...
Parameter combination 1 average validation loss: 0.0007 cross val losses: [0.0006388672045432031, 0.0006776363006792963, 0.0008181043085642159, 0.0004806887009181082, 0.0007238325197249651]
Testing parameter combination 2/16: (32, 64, 16, 20)
Training fold 1/5...
Training fold 2/5...
Training fold 3/5...
Training fold 4/5...
Training fold 5/5...
Parameter combination 2 average validation loss: 0.0007 cross val losses: [0.0005793540622107685, 0.0006682360544800758, 0.0009619655320420861, 0.0005251574912108481, 0.0008620915468782187]
Testing parameter combination 3/16: (32, 64, 32, 10)
Training fold 1/5...
Training fold 2/5...
Training fold 3/5...
Training fold 4/5...
Training fold 5/5...
Parameter combination 3 average validation loss: 0.0008 cross val losses: [0.0009896669071167707, 0.0007530309958383441, 0.0008342136279679835, 0.0005220957682467997, 0.0006884207832626998]
Testing parameter combination 

In [9]:
# Unpack best parameters
rnn_units, dense_units, batch_size, epochs= best_params

# Build the final model
final_model = build_model(SEQ_LENGTH, X_train.shape[-1], rnn_units, dense_units)

# Train the model
history = final_model.fit(
    X_train, y_train,
    epochs=epochs,
    batch_size=batch_size,
    validation_data=(X_valid, y_valid),
    callbacks=[early_stopping, tensorboard_callback],
    verbose=1
)

Epoch 1/10
1497/1497 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - loss: 0.0162 - mae: 0.0431 - val_loss: 0.0018 - val_mae: 0.0164
Epoch 2/10
1497/1497 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - loss: 7.7283e-04 - mae: 0.0160 - val_loss: 0.0017 - val_mae: 0.0179
Epoch 3/10
1497/1497 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - loss: 7.3333e-04 - mae: 0.0145 - val_loss: 0.0016 - val_mae: 0.0151
Epoch 4/10
1497/1497 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - loss: 6.7991e-04 - mae: 0.0142 - val_loss: 0.0020 - val_mae: 0.0217
Epoch 5/10
1497/1497 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - loss: 6.2001e-04 - mae: 0.0136 - val_loss: 0.0016 - val_mae: 0.0173


In [10]:
# Evaluate the model on the test set
test_loss, test_mae = final_model.evaluate(X_test, y_test, verbose=1)
print(f"Test loss: {test_loss:.4f}, Test MAE: {test_mae:.4f}")


514/514 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0014 - mae: 0.0155
Test loss: 0.0013, Test MAE: 0.0155


In [11]:
# Generate Bach-like music using the trained model
def generate_music(model, seed_sequence, num_steps=16):
    generated = seed_sequence.copy()
    for _ in range(num_steps):
        # Prepare input for the model
        input_seq = generated[-SEQ_LENGTH:].reshape(1, SEQ_LENGTH, -1)
        # Predict the next step
        next_step = model.predict(input_seq)
        generated = np.vstack((generated, next_step))  # Append the new step
    return generated

# Use the first test sequence as the seed
seed_sequence = X_test[0]
steps = 16
generated_music = generate_music(final_model, seed_sequence, num_steps=steps)

# Denormalize and convert to integers
generated_music = (generated_music * max_value).astype(int)
initial_music = (seed_sequence * max_value).astype(int)
print("Generated music shape:", generated_music.shape)
print(initial_music)
print("\tVVVVVV")
print(generated_music[-steps:])


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 203ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
Generated music shape: (48, 4)
[[65 60 57 53]
 [65 60 57 53]
 [65 60 57 53]
 [65 60 57 53]
 [72 60 55 52]
 [72 60 55 52]
 [70 60 55 52]
 [70 60 55 52]
 [69 60 53 53]
 [69 60 53 53]
 [67 60 55 52]
 [67 60 55 52]
 [65 62 57 50]
 [65 62 57 50]
 [65 64 58 50]
 [65 64 58 50]
 [72 65 60 45]
 [72 65 60 45]
 [72 65 60 45]
 [72 65 60 45]
 [74 65 60 46]
 [74 65 60 46]
 [74 65 58 48]
 [74 65 57 48]
